# Assignment 2: Language Modelling in Hangman

Student Name: Youngsun Kim

Student ID: 1022880

## General info

<b>Due date</b>: Tuesday, 6 April 2021 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 8% of mark for class (with 7% on correctness + 1% on quality and efficiency of your code)

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/121115/pages/using-jupyter-notebook-and-python?module_item_id=2681264) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. We recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.

<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each question is worth is explicitly given. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

# Overview

In this homework, you'll be creating an 'artificial intelligence' player for the classic Hangman word guessing game. You will need to implement several different automatic strategies based on character-level n-gram language models. Your objective is to create an automatic player which makes the fewest mistakes.

# The Hangman Game (7 marks)

**Instructions**: The <a href="https://en.wikipedia.org/wiki/Hangman_(game)">Hangman game</a> is a simple game whereby one person thinks of a word, which they keep secret from their opponent, who tries to guess the word one character at a time. The game ends when the opponent makes more than a fixed number of incorrect guesses, or they figure out the secret word before then (in which case they *win*). 

Here's a simple version of the game. **No implementation is needed.**

In [1]:
def hangman(secret_word, guesser, max_mistakes=8, verbose=True, **guesser_args):
    """
        This function plays the hangman game with the provided guesser and returns the number of incorrect guesses. 
        
        secret_word: a string of lower-case alphabetic characters, i.e., the answer to the game
        guesser: a function which guesses the next character at each stage in the game
            The function takes a:
                mask: what is known of the word, as a string with _ denoting an unknown character
                guessed: the set of characters which already been guessed in the game
                guesser_args: additional (optional) keyword arguments, i.e., name=value
        max_mistakes: limit on length of game, in terms of number of allowed mistakes
        verbose: silent or verbose diagnostic prints
        guesser_args: keyword arguments to pass directly to the guesser function
    """
    secret_word = secret_word.lower()
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, **guesser_args)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            guessed.add(guess)
            if guess in secret_word and len(guess) == 1:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if len(guess) != 1:
                    print('Please guess with only 1 character.')
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes

Here is a human guesser allowing interactive play.

In [2]:
def human(mask, guessed, **kwargs):
    """
    This is a simple function for manual play.
    """
    print('\nEnter your guess:')
    return input().lower().strip()

If you want to play hangman interactively, please set `interactive` to `True`. When submitting your solution, set to `False` so we can automatically run the whole notebook using `Run All`.

In [3]:
interactive = False

<b>For your testing:</b>

You can play the game interactively using the following command:

In [5]:
if interactive:
    hangman('whatever', human, 8, True)

### Question 1 (1.0 mark)

**Instructions**: We will use the words in NLTK's Brown corpus for training an artificial intelligence guessing algorithm, and for evaluating the quality of the algorithm.

Your first task is to compute the number of **unique word types** occurring in the Brown corpus, using `nltk.corpus.brown` and the `words` method, and select only words that are **entirely comprised of alphabetic characters**. You should also **lowercase the words**. Finally, randomly shuffle (`numpy.random.shuffle`) this collection of word types, and split them into disjoint training and testing sets. The test set should contain 1000 word types, and the rest should be in the  training set. Note that we are intentionally making the hangman game hard, as the AI will need to cope with test words that it has not seen before, hence it will need to learn generalisable patterns of characters to make reasonable predictions.

**Task**: Collect all unique word types from the Brown corpus, and produce `training_set` and `test_set`, 2 lists that contain 2 disjointed sets of words. Both `training_set` and `test_set` should be a python `list` (as initialised in the code). `test_set` must contain exactly 1000 word types.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [6]:
from nltk.corpus import brown
import numpy as np

np.random.seed(1)

# training_set stores the rest word types for training
training_set = []
# test_set stores 1000 word types for testing
test_set = []


###
# Your answer BEGINS HERE
###
import nltk
nltk.download('brown')
words=brown.words()
clean_words = list(filter(lambda x: x.isalpha(), words))
clean_words_lower = [t.lower() for t in clean_words]
unique_clean_word=np.unique(clean_words_lower)
np.random.shuffle(unique_clean_word)
training_set, test_set = unique_clean_word[1000:], unique_clean_word[:1000]

###
# Your answer ENDS HERE
###

print("Number of word types in test =", len(test_set))
print("Number of word types in train =", len(training_set))


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\Young\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


Number of word types in test = 1000
Number of word types in train = 39234


<b>For your testing:</b>

In [7]:
assert(len(training_set) > 35000 and len(training_set) < 45000)
assert(len(test_set) == 1000)

**Play the game**:

Let's see how good you are at this game! Try to guess a random word from the test set. It is surprisingly difficult (and addictive)! Don't forget to set `interactive = True`.

In [10]:
#play hangman using random words from test set
#interactive=True
if interactive:
    hangman(np.random.choice(test_set), human, 8, True)

2### Question 2 (1.0 mark)

**Instructions**: To set a baseline, your first AI attempt will be a trivial **random method**. For this you should implement a guessing method, similar to the `human` method above, i.e., using the same input arguments and returning a character. Your method should randomly choose a character from the range `a ... z` after excluding the characters that have already been guessed in the current game (all subsequent AI approaches should also exclude previous guesses).

To help you measure the performance of this (and later) guesser, a `test_guesser` method that takes a guesser and measures the average number of incorrect guesses made over all the words in the `test_set` is provided to you. 

**Task**: Complete the `random_guesser` method. It should return a random character from the English alphabets.

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [11]:
def test_guesser(guesser, test=test_set):
    """
        This function takes a guesser and measures the average number of incorrect guesses made over all the words in the test_set. 
    """
    total = 0
    for word in test:
        total += hangman(word, guesser, 26, False)
    return total / float(len(test))

In [12]:
def random_guesser(mask, guessed, **kwargs):
    ###
    # Your answer BEGINS HERE
    ###
    import string
    lower_alphabet = set(list(string.ascii_lowercase))
    remain_lower_alphabet = list(lower_alphabet - guessed)
    return np.random.choice(remain_lower_alphabet)
    ###
    # Your answer ENDS HERE
    ###
random_word=np.random.choice(test_set)
print("Guessing word =", random_word)
print("Number of mistakes made by the random guesser =", hangman(random_word, random_guesser, 26, False))

result = test_guesser(random_guesser)
print("\nTesting the random guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Guessing word = mighty
Number of mistakes made by the random guesser = 19

Testing the random guesser using every word in test set
Average number of incorrect guesses:  16.648


<b>For your testing:</b>

In [13]:
assert(result > 10 and result < 20)

### Question 3 (1.0 mark)

**Instructions:** As your first real AI, you should train a **unigram language model** over the training set. This requires you to find the frequencies of characters over all training words. Using this model, you should write a guesser that returns the character with the highest probability. Remember to exclude already guessed characters. 

**Task**: Collect the frequencies of characters and store them in `unigram_counts` (use the first answer space). Complete the `unigram_guesser` method. Note that it takes `unigram_counts` as an additional argument (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [14]:
unigram_counts = None

###
# Your answer BEGINS HERE
###
char_count={}
for words in training_set:
  for keys in words:
    if keys in char_count:
      char_count[keys]=char_count[keys]+1
    else:
      char_count[keys]=1
unigram_counts=char_count

# ###
# # Your answer ENDS HERE
# ###

def unigram_guesser(mask, guessed, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###
    non_repeated_count = dict((k, unigram_counts[k]) for k in unigram_counts if k not in guessed)
    total_count = float(sum(non_repeated_count.values()))
    for w in non_repeated_count:
          non_repeated_count[w] /= total_count

    max_count_char = max(non_repeated_count, key=non_repeated_count.get)
    return max_count_char
#     ###
#     # Your answer ENDS HERE
#     ###


result = test_guesser(unigram_guesser)
print("Testing the unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the unigram guesser using every word in test set
Average number of incorrect guesses:  10.333


<b>For your testing:</b>

In [15]:
assert(result > 5 and result < 15)

### Question 4 (1.0 mark)

**Instructions:** The length of the secret word is an important clue that we might exploit. Different lengths tend to have different distributions over characters, e.g., short words are less likely to have suffixes or prefixes. You should incorporate this idea by conditioning the unigram model on the length of the secret word, i.e.,  having a **different unigram model for each length**. You will need to be a little careful at test time, to be robust to the situation that you encounter a word length that you didn't see in training. In such a case, your method should behave like the previous `unigram_guesser` in Question 3 (i.e., it guesses characters based on unigram frequencies, unconditioned by the word length).

**Task**: Collect the frequencies of characters conditioned on the word length and store them in `unigram_counts_by_length` (use the first answer space). Complete the `unigram_length_guesser` method (use the second answer space).

**Check**: Use the assertion statements in <b>"For your testing"</b> below for the expected output.

In [16]:
unigram_counts_by_length = None
from collections import defaultdict

###
# Your answer BEGINS HERE
###
char_count=defaultdict(lambda : defaultdict(lambda : defaultdict(int)))
for words in training_set:
  for keys in words:
    if len(words) in char_count:
      if keys in char_count[len(words)]:
        char_count[len(words)][keys]=char_count[len(words)][keys]+1
      else:
        char_count[len(words)][keys]=1
    else:
      char_count[len(words)][keys]=1


unigram_counts_by_length=char_count

# ###
# # Your answer ENDS HERE
# ###

def unigram_length_guesser(mask, guessed, unigram_counts_by_length=unigram_counts_by_length, unigram_counts=unigram_counts):
    ###
    # Your answer BEGINS HERE
    ###
    if len(mask) in unigram_counts_by_length.keys():
      non_repeated_count=dict((k, unigram_counts_by_length[len(mask)][k]) for k in unigram_counts_by_length[len(mask)] if k not in guessed)

    else:
      non_repeated_count = dict((k, unigram_counts[k]) for k in unigram_counts if k not in guessed)

    total_count = float(sum(non_repeated_count.values()))
    for w in non_repeated_count:
          non_repeated_count[w] /= total_count

    

    max_count_char = max(non_repeated_count, key=non_repeated_count.get)
    return max_count_char



    ###
    # Your answer ENDS HERE
    ###

result = test_guesser(unigram_length_guesser)
print("Testing the length-conditioned unigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the length-conditioned unigram guesser using every word in test set
Average number of incorrect guesses:  10.211


<b>For your testing:</b>

In [17]:
assert(result > 5 and result < 15)

### Question 5 (1.0 mark)

**Instructions:** Now for the next challenge, you'll build a **bigram language model** over characters. The order of characters is obviously important, yet this wasn't incorporated in any of the above models. Knowing that the word has the sequence `n _ s s` is a pretty strong clue that the missing character might be `e`. Similarly the distribution over characters that start or end a word are highly biased (e.g., toward common prefixes and suffixes, like *un-*, *-ed* and *-ly*).

The task here is to develop a bigram language model over characters, and train it over the training words. Remember to be careful when handling the start of each word properly, e.g., by padding with a special starting symbol such as `$`. Do we also need a special ending symbol? That's for you to decide.

Your bigram guesser should apply your language model to each blank position in the secret word by using its left context character. For example, in the partial word `e _ c _ b _ _` we know the left context for the first three blanks, but have no known left context for the last blank. In the case for the last blank, you should revert to using a unigram language model (since there's no context for us to use the bigram model). You should sum up the probability distribution (over all alphabets from <i>a</i> to <i>z</i>) for the 4 blanks, and select the alphabet with the highest probability that hasn't been guessed.

**Task**: Collect frequency counts that are necessary for building a bigram language model and store them in bigram_counts; feel free to add new objects if needed (use the first answer space). Complete the `bigram_guesser` method (use the second answer space). Note that the method currently only has one additional argument (`bigram_counts`), but you are free to add additional arguments.

In [21]:
bigram_counts = None
from collections import defaultdict 

###
# Your answer BEGINS HERE
###
unigram = defaultdict(int)
bigram_counts = defaultdict(lambda: defaultdict(int))
        
# go through each word in the train
for word in training_set:
  # check each letter in the train and update the n-gram
  for i in range(len(word) - 1):
    bigram_counts[word[i]][word[i+1]] += 1

  bigram_counts['$'][word[0]]+=1

  # fill out unigrams
  for letter in set(word):
    unigram[letter] += 1
    

###
# Your answer ENDS HERE
###
    

def bigram_guesser(mask, guessed, bigram_counts=bigram_counts,unigram=unigram): # add extra arguments if needed
    ###
    # Your answer BEGINS HERE
    ###
    import string
    letter_set = list(string.ascii_lowercase)
    probabilities = [0] * len(letter_set)

    probs = [0] * len(letter_set)
    total_count = 0
    letter_count = [0] * len(letter_set)
    mask=['$']+mask
    for i in range(len(mask)-1):
      if mask[i] != '_' and mask[i+1] == '_':
                anchor_letter = mask[i]
                # calculate occurences of "anchor_letter blank" and each letter not guessed yet
                for j, letter in enumerate(letter_set):
                    if bigram_counts[anchor_letter][letter] > 0 and letter not in guessed:
                        total_count += bigram_counts[anchor_letter][letter]
                        letter_count[j] += bigram_counts[anchor_letter][letter]
    if total_count > 0:
            for i in range(len(letter_set)):
                probs[i] = letter_count[i] / total_count


    for i, p in enumerate(probabilities):
            probabilities[i] = p + probs[i]
            
    probs_uni = [0] * len(letter_set)
    total_count = 0
    letter_count = [0] * len(letter_set)

    # traverse the mask and find blank spaces
    for i in range(len(mask)):
      if mask[i] == '_':
                                
                # calculate occurences of pattern and each letter not guessed yet
                for j, letter in enumerate(letter_set):
                    if unigram[letter] > 0 and letter not in guessed:
                        total_count += unigram[letter]
                        letter_count[j] += unigram[letter]

    if total_count > 0:
      for i in range(len(letter_set)):
        probs_uni[i] = letter_count[i] / total_count
    for i, p in enumerate(probabilities):
      probabilities[i] = p + probs_uni[i] 

    # adjust probabilities so they sum to one
    final_probs = defaultdict(int)
    if sum(probabilities) > 0:
      for i in range(len(probabilities)):
        final_probs[letter_set[i]] = float(probabilities[i] / sum(probabilities))


    max_count_char = max(final_probs, key=final_probs.get)
    

    return max_count_char



    

    ###
    # Your answer ENDS HERE
    ###


result = test_guesser(bigram_guesser)
print("Testing the bigram guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing the bigram guesser using every word in test set
Average number of incorrect guesses:  8.877


### Question 6 (1.5 mark)

**Instructions:** You should try to develop a more effective AI for hangman. Feel free to engage your creativity here! Possibilities include better conditioning on the length of the word, fancier smoothing methods, ngram models and bidirectional models (lecture 8). Have fun!

You will be marked based on the performance of your AI model, using a pre-made training and test set (created using a secret seed). Let x be the average number of mistakes in the test set, you will score:
* 1.5 mark if x < 8.0
* 1.0 mark if 8.0 <= x < 8.5
* 0.5 mark if 8.5 <= x < 8.8
* 0.0 mark if x >= 8.8

Note: when testing your AI model's performance, you may want to consider trying different training/test splits (using different seeds) to have a better understanding of its performance, as there will be some variance to its performance depending on the training/test split.

**Task** Complete the `my_amazing_ai_guesser` method, which implements a better language model for hangman.

In [23]:
###
# Your answer BEGINS HERE
###
unigram = defaultdict(lambda: defaultdict(int))
bi_gram = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
tri_gram = defaultdict(lambda: defaultdict(lambda: defaultdict(int)))
four_gram = defaultdict(lambda:defaultdict(lambda: defaultdict(lambda: defaultdict(int))))
five_gram = defaultdict(lambda: defaultdict(lambda:defaultdict(lambda: defaultdict(lambda: defaultdict(int)))))

# foreach word in the training set
for word in training_set:
  # check each letter in the tranning set and update the n-gram
  for i in range(len(word) - 4):
    bi_gram[len(word)][word[i]][word[i+1]] += 1
    tri_gram[word[i]][word[i+1]][word[i+2]] += 1
    four_gram[word[i]][word[i+1]][word[i+2]][word[i+3]] += 1
    five_gram[word[i]][word[i+1]][word[i+2]][word[i+3]][word[i+4]] += 1
  i = len(word) - 4
  if len(word) == 2:
    bi_gram[len(word)][word[0]][word[1]] += 1
  elif len(word) == 3:
    bi_gram[len(word)][word[0]][word[1]] += 1
    bi_gram[len(word)][word[1]][word[2]] += 1
    tri_gram[word[0]][word[1]][word[2]] += 1
  elif len(word) >= 4:
    bi_gram[len(word)][word[i]][word[i+1]] += 1
    bi_gram[len(word)][word[i+1]][word[i+2]] += 1
    bi_gram[len(word)][word[i+2]][word[i+3]] += 1
    tri_gram[word[i]][word[i+1]][word[i+2]] += 1
    tri_gram[word[i+1]][word[i+2]][word[i+3]] += 1
    four_gram[word[i]][word[i+1]][word[i+2]][word[i+3]] += 1
  # unigrams
  for letter in set(word):
    unigram[len(word)][letter] += 1


def my_amazing_ai_guesser(mask, guessed,unigram=unigram,bigram=bi_gram,trigram=tri_gram,fourgram=four_gram,fivegram=five_gram):
  import string
  letter_set = list(string.ascii_lowercase)
  probabilities = [0] * len(letter_set)

  # vector of probabilities for each letter in five gram
  probs = [0] * len(letter_set)
  total_count = 0
  letter_count = [0] * len(letter_set)
  word=mask
  # traverse the mask and find patterns that have five consecutive letters where one of them is blank
  for i in range(len(word) - 4):
    # case 1: "letter letter letter letter blank"
    if word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] == '_':
      anchor_letter_1 = word[i]
      anchor_letter_2 = word[i+1]
      anchor_letter_3 = word[i+2]
      anchor_letter_4 = word[i+3]
    # calculate occurences of "anchor_letter_1 anchor_letter_2 anchor_letter_3 anchor_letter_4 blank" and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4][letter] > 0 and letter not in guessed:
          total_count += fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4][letter]
          letter_count[j] += fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4][letter]
  
    # case 2: "letter letter letter blank letter"
    elif word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] == '_' and word[i+4] != '_':
      anchor_letter_1 = word[i]
      anchor_letter_2 = word[i+1]
      anchor_letter_3 = word[i+2]
      anchor_letter_4 = word[i+4]
      # calculate occurences of "anchor_letter_1 anchor_letter_2 anchor_letter_3 blank anchor_letter_4 " and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter][anchor_letter_4] > 0 and letter not in guessed:
          total_count += fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter][anchor_letter_4]
          letter_count[j] += fivegram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter][anchor_letter_4]
               
      # case 3: letter letter blank letter letter
    elif word[i] != '_' and word[i+1] != '_' and word[i+2] == '_' and word[i+3] != '_' and word[i+4] != '_':
      anchor_letter_1 = word[i]
      anchor_letter_2 = word[i+1]
      anchor_letter_3 = word[i+3]
      anchor_letter_4 = word[i+4]
      # calculate occurences of "anchor_letter_1 anchor_letter_2 blank anchor_letter_3 anchor_letter_4" and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if fivegram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3][anchor_letter_4] > 0 and letter not in guessed:
          total_count += fivegram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3][anchor_letter_4]
          letter_count[j] += fivegram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3][anchor_letter_4]
               
    # case 4: letter blank letter letter letter
    elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] != '_':
      anchor_letter_1 = word[i]
      anchor_letter_2 = word[i+2]
      anchor_letter_3 = word[i+3]
      anchor_letter_4 = word[i+4]
    # calculate occurences of "anchor_letter_1 blank  anchor_letter_2 anchor_letter_3 anchor_letter_4" and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if fivegram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3][anchor_letter_4] > 0 and letter not in guessed:
          total_count += fivegram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3][anchor_letter_4]
          letter_count[j] += fivegram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3][anchor_letter_4]
        
    # case 5: blank letter letter letter letter
    elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_' and word[i+4] != '_':
      anchor_letter_1 = word[i+1]
      anchor_letter_2 = word[i+2]
      anchor_letter_3 = word[i+3]
      anchor_letter_4 = word[i+4]
      # calculate occurences of "blank anchor_letter_1 anchor_letter_2 anchor_letter_3 anchor_letter_4 " and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if fivegram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4] > 0 and letter not in guessed:
          total_count += fivegram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4]
          letter_count[j] += fivegram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3][anchor_letter_4]
        
  # calculate the probabilities of each letter 
  if total_count > 0:
    for i in range(len(letter_set)):
      probs[i] = letter_count[i] / total_count
  for i, p in enumerate(probabilities):
    probabilities[i] = p + probs[i] * (0.40)

  probs = [0] * len(letter_set)
  total_count = 0
  letter_count = [0] * len(letter_set)

  for i in range(len(word) - 3):
    # case 1: "letter letter letter blank"
    if word[i] != '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] == '_':
      anchor_letter_1 = word[i]
      anchor_letter_2 = word[i+1]
      anchor_letter_3 = word[i+2]
      # calculate occurences of "anchor_letter_1 anchor_letter_2 anchor_letter_3 blank" and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if fourgram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter] > 0 and letter not in guessed:
          total_count += fourgram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter]
          letter_count[j] += fourgram[anchor_letter_1][anchor_letter_2][anchor_letter_3][letter]
      # case 2: "letter letter blank letter"
    elif word[i] != '_' and word[i+1] != '_' and word[i+2] == '_' and word[i+3] != '_':
      anchor_letter_1 = word[i]
      anchor_letter_2 = word[i+1]
      anchor_letter_3 = word[i+3]
                
      # calculate occurences of "anchor_letter_1 anchor_letter_2 blank anchor_letter_3" and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if fourgram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3] > 0 and letter not in guessed:
          total_count += fourgram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3]
          letter_count[j] += fourgram[anchor_letter_1][anchor_letter_2][letter][anchor_letter_3]

    # case 3: letter blank letter letter
    elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_' and word[i+3] != '_':
      anchor_letter_1 = word[i]
      anchor_letter_2 = word[i+2]
      anchor_letter_3 = word[i+3]
      # calculate occurences of "anchor_letter_1 blank anchor_letter_2 anchor_letter_3" and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if fourgram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3] > 0 and letter not in guessed:
          total_count += fourgram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3]
          letter_count[j] += fourgram[anchor_letter_1][letter][anchor_letter_2][anchor_letter_3]
               
    # case 4: blank letter letter letter
    elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_' and word[i+3] != '_':
      anchor_letter_1 = word[i+1]
      anchor_letter_2 = word[i+2]
      anchor_letter_3 = word[i+3]
      # calculate occurences of "blank anchor_letter_1 anchor_letter_2 anchor_letter_3" and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if fourgram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3] > 0 and letter not in guessed:
          total_count += fourgram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3]
          letter_count[j] += fourgram[letter][anchor_letter_1][anchor_letter_2][anchor_letter_3]

  # calculate the probabilities of each letter 
  if total_count > 0:
    for i in range(len(letter_set)):
      probs[i] = letter_count[i] / total_count

  for i, p in enumerate(probabilities):
          probabilities[i] = p + probs[i] * (0.25)

  probs = [0] * len(letter_set)
  total_count = 0
  letter_count = [0] * len(letter_set)

  for i in range(len(word) - 2):
    # case 1: "letter letter blank"
    if word[i] != '_' and word[i+1] != '_' and word[i+2] == '_':
      anchor_letter_1 = word[i]
      anchor_letter_2 = word[i+1]
      # calculate occurences of "anchor_letter_1 anchor_letter_2 blank" and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if trigram[anchor_letter_1][anchor_letter_2][letter] > 0 and letter not in guessed:
          total_count += trigram[anchor_letter_1][anchor_letter_2][letter]
          letter_count[j] += trigram[anchor_letter_1][anchor_letter_2][letter]
    # case 2: "letter blank letter"
    elif word[i] != '_' and word[i+1] == '_' and word[i+2] != '_':
      anchor_letter_1 = word[i]
      anchor_letter_2 = word[i+2]
                
      # calculate occurences of "anchor_letter_1 blank anchor_letter_2" and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if trigram[anchor_letter_1][letter][anchor_letter_2] > 0 and letter not in guessed:
          total_count += trigram[anchor_letter_1][letter][anchor_letter_2]
          letter_count[j] += trigram[anchor_letter_1][letter][anchor_letter_2]
               
    # case 3: blank letter letter
    elif word[i] == '_' and word[i+1] != '_' and word[i+2] != '_':
      anchor_letter_1 = word[i+1]
      anchor_letter_2 = word[i+2]
                
      # calculate occurences of "blank anchor_letter_1 anchor_letter_2" and for each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if trigram[letter][anchor_letter_1][anchor_letter_2] > 0 and letter not in guessed:
          total_count += trigram[letter][anchor_letter_1][anchor_letter_2]
          letter_count[j] += trigram[letter][anchor_letter_1][anchor_letter_2]
  # calculate the probabilities of each letter 
  if total_count > 0:
    for i in range(len(letter_set)):
      probs[i] = letter_count[i] / total_count

  for i, p in enumerate(probabilities):
          probabilities[i] = p + probs[i] * (0.20)

  probs = [0] * len(letter_set)
  total_count = 0
  letter_count = [0] * len(letter_set)

  for i in range(len(word) - 1):
    # case 1: "letter blank"
    if word[i] != '_' and word[i+1] == '_':
      anchor_letter = word[i]
    # calculate occurences of "anchor_letter blank" and each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if bigram[len(word)][anchor_letter][letter] > 0 and letter not in guessed:
          total_count += bigram[len(word)][anchor_letter][letter]
          letter_count[j] += bigram[len(word)][anchor_letter][letter]
                            
    # case 2: "blank letter"
    elif word[i] == '_' and word[i+1]!= '_':
      anchor_letter = word[i+1]
                
      # calculate occurences of "blank anchor_letter" and each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if bigram[len(word)][letter][anchor_letter] > 0 and letter not in guessed:
          total_count += bigram[len(word)][letter][anchor_letter]
          letter_count[j] += bigram[len(word)][letter][anchor_letter]
  # calculate the probabilities of each letter 
  if total_count > 0:
    for i in range(len(letter_set)):
      probs[i] = letter_count[i] / total_count

  for i, p in enumerate(probabilities):
          probabilities[i] = p + probs[i] * (0.10)

  probs = [0] * len(letter_set)
  total_count = 0
  letter_count = [0] * len(letter_set)

  for i in range(len(word)):
    # case 1: "letter blank"
    if word[i] == '_':
      # calculate occurences of pattern and each letter not guessed yet
      for j, letter in enumerate(letter_set):
        if unigram[len(word)][letter] > 0 and letter not in guessed:
          total_count += unigram[len(word)][letter]
          letter_count[j] += unigram[len(word)][letter]

  # calculate the probabilities of each letter 
  if total_count > 0:
    for i in range(len(letter_set)):
      probs[i] = letter_count[i] / total_count

  for i, p in enumerate(probabilities):
          probabilities[i] = p + probs[i] * (0.10)

  # adjust probabilities so they sum to one
  final_probs = defaultdict(int)
  if sum(probabilities) > 0:
    for i in range(len(probabilities)):
      final_probs[letter_set[i]] = float(probabilities[i] / sum(probabilities))


  max_count_char = max(final_probs, key=final_probs.get)



  return max_count_char

###
# Your answer ENDS HERE
###

result = test_guesser(my_amazing_ai_guesser)
print("Testing my amazing AI guesser using every word in test set")
print("Average number of incorrect guesses: ", result)

Testing my amazing AI guesser using every word in test set
Average number of incorrect guesses:  5.935


### Question 7 (0.5 mark)

**Instructions:** Explain your approach and discuss your result below. Please keep your explanation to a short paragraph.

##### Your answer BEGINS HERE


To determine the best guess, this approach uses n-gram counts of letters. I implemented n-grams from 1 (unigram), 2 (bigram) up to 5 (five-gram). 'Five' is the chosen cutoff because most of the 6 and above letter sequences begin to be just a chain of smaller sequences. 

For all words of test data, I used a few steps as below.
 
First, all blanks (underscores separated by spaces) are fed to the function to guess the first letter. For example, "hangman" is converted to "_ _ _ _ _ _ _". Any correctly guessed letter will be blank, and this new string is passed for the next guess. A new string passed from the previous stage find any potential n-grams. For example, a unigram can be anywhere if there is any single blank space. A bigram can be either 'letter-blank' and 'blank-letter' such as '_ h' or 'h _'. Therefore a single blank space can be multiple n-grams. If the current status of the word is "h a _ g m an", the blank space is a unigram, bigram (for both "a " and " g"), trigram ("h a ", "a _ g", " g m"), four-gram and five-gram.
For each blank, a vector of probabilities for each letter not yet guessed is tabulated by calculating the frequencies of each n-gram, with more weight given to larger n. The letter with the highest probability is taken as a guess word. With our training data, the average number of incorrect guess showed 5.935 which is a fairly good result.



##### Your answer ENDS HERE